In [1]:
!pip install tensorflow==2.16.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires ml_dtypes>=0.4.0, but you have ml-dtypes 0.3.2 which is incomp

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

# **Step 01: Load Dataset**

In [3]:
(train_dataset, val_dataset, _), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.RYZALF_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


# **Step 02: Preprocessing**

In [4]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

In [5]:

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    return image / 255.0, label

In [6]:
train_dataset = train_dataset.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess).batch(BATCH_SIZE)

# **Step 03: Train Model**

In [7]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(150, 150, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

<ipython-input-7-1e7f2c719d97>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# Train
model.fit(train_dataset, epochs=5, validation_data=val_dataset)

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 364s 613ms/step - accuracy: 0.9363 - loss: 0.1716 - val_accuracy: 0.9673 - val_loss: 0.0872
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 354s 608ms/step - accuracy: 0.9629 - loss: 0.0921 - val_accuracy: 0.9647 - val_loss: 0.0851
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 386s 615ms/step - accuracy: 0.9670 - loss: 0.0824 - val_accuracy: 0.9656 - val_loss: 0.0934
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 371s 636ms/step - accuracy: 0.9701 - loss: 0.0756 - val_accuracy: 0.9673 - val_loss: 0.0840
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 383s 638ms/step - accuracy: 0.9730 - loss: 0.0690 - val_accuracy: 0.9682 - val_loss: 0.0806


# **Step 04: Save Model**

In [10]:
# Save in Keras 3 compatible format
model.save('cats_dogs_model.keras')

In [11]:
# Load the saved model
loaded_model = tf.keras.models.load_model('cats_dogs_model.keras')

# **Step 05: Evaluate Performance**

In [12]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = loaded_model.evaluate(val_dataset)

73/73 ━━━━━━━━━━━━━━━━━━━━ 45s 554ms/step - accuracy: 0.9693 - loss: 0.0825


In [13]:
# Print the results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Test Loss: 0.08062633126974106
Test Accuracy: 0.9681857228279114
